In [1]:
#http://anytree.readthedocs.io/en/latest/api.html
import anytree 

udo = anytree.Node("Udo")
marc = anytree.Node("Marc", parent=udo)
lian = anytree.Node("Lian", parent=marc)
dan = anytree.Node("Dan", parent=udo)
jet = anytree.Node("Jet", parent=dan)
jan = anytree.Node("Jan", parent=dan)
joe = anytree.Node("Joe", parent=dan)

In [2]:
class FWrapper:
    def __init__(self,function,child_count,name):
        self.function=function
        self.child_count=child_count
        self.name=name

class Node(anytree.NodeMixin):
    def __init__(self,fw,children,parent=None):
        super(Node, self).__init__()
        self.function=fw.function
        self.name=fw.name
        self.children=children
        self.parent=parent
        self.visited = False
    def evaluate(self,inp):    
        results=[n.evaluate(inp) for n in self.children]
        return self.function(results)
    
    def display(self,indent=0):
        print((' '*indent)+self.name)
    
        for c in self.children:
            c.display(indent+1)

class VarNode(anytree.NodeMixin):
    def __init__(self, idx, parent=None):
        super(VarNode, self).__init__()
        self.idx = idx
        self.name="v{}".format(idx)
        self.parent = parent
        self.visited = False
    def evaluate(self,inp):
        return inp[self.idx]
    def display(self,indent=0):
        print('%sv%d' % (' '*indent,self.idx))
    
    
class ConstNode(anytree.NodeMixin):
    def __init__(self, value, parent=None):
        super(ConstNode, self).__init__()
        self.value= value
        self.name= "c{}".format(value)
        self.parent = parent
        self.visited = False
    def evaluate(self, inp):
        return self.value
    def display(self,indent=0):
        print('%sc%d' % (' '*indent,self.value))

In [3]:
add_w=FWrapper(lambda l:l[0]+l[1],2,'add')
sub_w=FWrapper(lambda l:l[0]-l[1],2,'sub') 
mul_w=FWrapper(lambda l:l[0]*l[1],2,'mul')

def if_func(l):
    if l[0]>0: return l[1]
    else: return l[2]

if_w = FWrapper(if_func,3,'if')

def gt_func(l):
    if l[0]>l[1]: return 1
    else: return 0
    
gt_w = FWrapper(gt_func,2,'gt')

def lt_func(l):
    if l[0]>l[1]: return 1
    else: return 0
    
lt_w = FWrapper(lt_func,2,'lt')

f_list=[add_w, sub_w, mul_w, if_w, gt_w, lt_w]

In [4]:
example_tree = Node(if_w,[
                    Node(gt_w,[VarNode(0),ConstNode(3)]),
                    Node(add_w,[VarNode(1),ConstNode(5)]),
                    Node(sub_w,[VarNode(1),ConstNode(2)]),
                    ]
               )

In [5]:
example_tree.evaluate([2,3])

1

In [6]:
#f(x,y) -> if x > 3: 
#            return y+5
#          else:
#            return y-2

example_tree.display()

if
 gt
  v0
  c3
 add
  v1
  c5
 sub
  v1
  c2


In [13]:
def DFS_print(tree):
    """
    Depth First Search
    """
    name = tree.name
    try:
        parent = tree.parent.name
    except:
        parent = 'root'
    children = [c.name for c in tree.children]
    if len(children)==0:
        children = ['leaf']
    print("name: {:5}  parent: {:5}  children: {}".format(name, parent, children))
    tree.visited = True
    for c in tree.children:
        if c.visited == False:
            DFS_print(c)

In [14]:
example_tree = Node(if_w,[
                    Node(gt_w,[VarNode(0),ConstNode(3)]),
                    Node(add_w,[VarNode(1),ConstNode(5)]),
                    Node(sub_w,[VarNode(1),ConstNode(2)]),
                    ]
               )

DFS_print(example_tree)

name: if     parent: root   children: ['gt', 'add', 'sub']
name: gt     parent: if     children: ['v0', 'c3']
name: v0     parent: gt     children: ['leaf']
name: c3     parent: gt     children: ['leaf']
name: add    parent: if     children: ['v1', 'c5']
name: v1     parent: add    children: ['leaf']
name: c5     parent: add    children: ['leaf']
name: sub    parent: if     children: ['v1', 'c2']
name: v1     parent: sub    children: ['leaf']
name: c2     parent: sub    children: ['leaf']


In [15]:
def tree_to_sequence(tree, seq=''):
    name = tree.name
    if len(tree.children)>0:
        name+=' ( '
    else:
        name+=' , '
    seq += name
    tree.visited = True
    for c in tree.children:
        if c.visited == False:
            seq += tree_to_sequence(c)
    if len(tree.children)>0:
        seq += ' ) , '
    return seq

In [16]:
example_tree = Node(if_w,[
                    Node(gt_w,[VarNode(0),ConstNode(3)]),
                    Node(add_w,[VarNode(1),ConstNode(5)]),
                    Node(sub_w,[VarNode(1),ConstNode(2)]),
                    ]
               )

tree_to_sequence(example_tree)

'if ( gt ( v0 , c3 ,  ) , add ( v1 , c5 ,  ) , sub ( v1 , c2 ,  ) ,  ) , '

In [17]:
example_tree = Node(if_w,[
                    Node(gt_w,[VarNode(0),ConstNode(3)]),
                    Node(add_w,[VarNode(1),ConstNode(5)]),
                    Node(sub_w,[VarNode(1),ConstNode(2)]),
                    ]
               )

def return_sequence(example_tree):
    sequence = tree_to_sequence(example_tree)
    sequence = sequence.split()
    for i, s in enumerate(sequence):
        if i<(len(sequence)-1) and sequence[i]==',' and sequence[i+1]==')':
            del sequence[i]
        if sequence[-1]==',':
            del sequence[-1]
    return ' '.join(sequence)

In [18]:
return_sequence(example_tree)

'if ( gt ( v0 , c3 ) , add ( v1 , c5 ) , sub ( v1 , c2 ) )'